**Script Updation_NLP**


Takes a script as an input.

Performs POS tagging.

Generates questions to the user from the specific nouns present in the script.

Modifies the script based on the answers provided by the user for each question.

Using sentence detection, it seperates each and every sentence from the modified script and gives the output as list which will be feeded to the diffuser model.





In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

Opens and reads a text file
Sentence detection

In [ ]:
with open("/content/Script _2.txt", "r") as file:
  text = file.read()
doc = nlp(text)
sents = list(doc.sents)
print("No. of Sentence in given prompt:",len(sents))

No. of Sentence in given prompt: 21


In [ ]:
for sent in sents:
    print(f"{sent}")

In a peaceful neighborhood, we find a curious cat and an energetic dog living next door to each other.
They lead separate lives, unaware of the incredible friendship that awaits them.
One sunny day, as the cat strolls through the neighborhood, it spots the dog playing with a ball in the backyard.
Intrigued, the cat approaches cautiously, and their eyes meet for the first time.

The cat and dog engage in a playful game of chase, running through the park and rolling in the grass.
Their energy and excitement bring laughter and joy to both.
As the days pass, the cat and dog embark on countless adventures together.
They explore hidden paths, climb trees, and discover new places, creating lasting memories.
During a stormy night, the cat seeks shelter under a porch, scared by the thunder.
The dog rushes to its side, providing comfort and reassurance.
Their bond strengthens as they offer support to one another.

The cat teaches the dog the art of stealth and graceful movements, while the dog s

POS tagging to find nouns and verbs

In [ ]:
l=[]

In [ ]:
for sent in range(len(sents)):
  d={"NOUN":[],"VERB":[]}
  for i in sents[sent]:
    if(i.pos_ == "NOUN" or i.pos_ == "VERB"):
      d[i.pos_].append(str(i))
  l.append(d)
  print("Script ",sent+1,":",l[sent])

Script  1 : {'NOUN': ['neighborhood', 'cat', 'dog', 'door'], 'VERB': ['find', 'living']}
Script  2 : {'NOUN': ['lives', 'friendship'], 'VERB': ['lead', 'awaits']}
Script  3 : {'NOUN': ['day', 'cat', 'neighborhood', 'dog', 'ball', 'backyard'], 'VERB': ['strolls', 'spots', 'playing']}
Script  4 : {'NOUN': ['cat', 'eyes', 'time'], 'VERB': ['Intrigued', 'approaches', 'meet']}
Script  5 : {'NOUN': ['cat', 'dog', 'engage', 'game', 'chase', 'park', 'grass'], 'VERB': ['running', 'rolling']}
Script  6 : {'NOUN': ['energy', 'excitement', 'laughter', 'joy'], 'VERB': ['bring']}
Script  7 : {'NOUN': ['days', 'cat', 'dog', 'embark', 'adventures'], 'VERB': ['pass']}
Script  8 : {'NOUN': ['paths', 'climb', 'trees', 'places', 'memories'], 'VERB': ['explore', 'discover', 'creating']}
Script  9 : {'NOUN': ['night', 'cat', 'shelter', 'porch', 'thunder'], 'VERB': ['seeks', 'scared']}
Script  10 : {'NOUN': ['dog', 'side', 'comfort', 'reassurance'], 'VERB': ['rushes', 'providing']}
Script  11 : {'NOUN': ['bo

Function to generate question, store answers and modify the script.

In [ ]:
def generate(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    sents = list(doc.sents)
    script = ""
    questions = []
    subjects = set()
    nouns = set()
    proper_nouns = set()
    for sent in range(len(sents)):
        d = {'subject': '', 'preceding_nouns': set()}
        subject_found = False
        for i in sents[sent]:
            if i.dep_ == "nsubj" and i.pos_ == "NOUN" and not subject_found and str(i) not in subjects:
                d['subject'] = str(i)
                subject_found = True
                subjects.add(str(i))
            elif i.pos_ == "NOUN" and str(i) != d['subject'] and str(i) not in nouns:
                if i.pos_ == "PROPN":
                    proper_nouns.add(str(i))
                else:
                    d['preceding_nouns'].add(str(i))
                nouns.add(str(i))
        if d['subject'] != '':
            script += "Script {}: Subject: {}\n".format(sent + 1, d['subject'])
            if d['preceding_nouns']:
                script += "Preceding Nouns: {}\n".format(", ".join(list(d['preceding_nouns'])))
            if proper_nouns:
                script += "Proper Nouns: {}\n".format(", ".join(list(proper_nouns)))

            if d['subject'] in nouns:
                question = "Describe the features of {}?".format(d['subject'])
            else:
                question = "Specify more details about {}?".format(d['subject'])
            questions.append(question)
        else:
            script += "Script {}: None\n".format(sent + 1)
    return script, questions

def g_answers(questions):
    answers = []
    for question in questions:
        answer = input(question + " ")
        answers.append(answer)
    return answers

def g_text(text, questions, answers):
    new_text = text
    for i, answer in enumerate(answers):
        noun = questions[i].split(' ')[-1].strip('?')
        new_text = new_text.replace(noun, f"{answer} {noun}")
    return new_text
script, questions = generate(text)
answers = g_answers(questions)
new_text = g_text(text, questions, answers)
print("\n")
print(new_text)

Describe the features of cat? white
Specify more details about eyes? blue
Specify more details about energy? high
Specify more details about days? past
Describe the features of dog? brown
Specify more details about bond? intact
Describe the features of time? current
Specify more details about Laughter? enormus


In a peaceful neighborhood, we find a curious white cat and an energetic brown dog living next door to each other. They lead separate lives, unaware of the incredible friendship that awaits them. One sunny day, as the white cat strolls through the neighborhood, it spots the brown dog playing with a ball in the backyard. Intrigued, the white cat approaches cautiously, and their blue eyes meet for the first current time.
The white cat and brown dog engage in a playful game of chase, running through the park and rolling in the grass. Their high energy and excitement bring laughter and joy to both. As the past days pass, the white cat and brown dog embark on countless adventures to

Sentence detection for modified text

In [ ]:
doc1=nlp(new_text)
sents1=list(doc1.sents)
for sent in sents1:
  print(f"{sent}")

In a peaceful neighborhood, we find a curious white cat and an energetic brown dog living next door to each other.
They lead separate lives, unaware of the incredible friendship that awaits them.
One sunny day, as the white cat strolls through the neighborhood, it spots the brown dog playing with a ball in the backyard.
Intrigued, the white cat approaches cautiously, and their blue eyes meet for the first current time.

The white cat and brown dog engage in a playful game of chase, running through the park and rolling in the grass.
Their high energy and excitement bring laughter and joy to both.
As the past days pass, the white cat and brown dog embark on countless adventures together.
They explore hidden paths, climb trees, and discover new places, creating lasting memories.
During a stormy night, the white cat seeks shelter under a porch, scared by the thunder.
The brown dog rushes to its side, providing comfort and reassurance.
Their intact bond strengthens as they offer support to 

In [ ]:
sentences = [sent.text for sent in doc1.sents]
print(sentences)

['In a peaceful neighborhood, we find a curious white cat and an energetic brown dog living next door to each other.', 'They lead separate lives, unaware of the incredible friendship that awaits them.', 'One sunny day, as the white cat strolls through the neighborhood, it spots the brown dog playing with a ball in the backyard.', 'Intrigued, the white cat approaches cautiously, and their blue eyes meet for the first current time.\n', 'The white cat and brown dog engage in a playful game of chase, running through the park and rolling in the grass.', 'Their high energy and excitement bring laughter and joy to both.', 'As the past days pass, the white cat and brown dog embark on countless adventures together.', 'They explore hidden paths, climb trees, and discover new places, creating lasting memories.', 'During a stormy night, the white cat seeks shelter under a porch, scared by the thunder.', 'The brown dog rushes to its side, providing comfort and reassurance.', 'Their intact bond stre